# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import t
from scipy.stats import norm
from scipy.stats import ttest_1samp


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [3]:
# Your code here:
data = pd.read_csv('./hypothesis-testing.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:

data.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
data.isnull().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:

data['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
# Your code here:
dep_empl = data[['Department','Name']]
dep_empl.groupby(['Department']).count()



,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [28]:
# Your code here:
data_hourly = data[['Salary or Hourly', 'Hourly Rate']]
filter_hourly = data['Salary or Hourly'] == 'Hourly'
test_series = data_hourly[filter_hourly]

std = np.std(test_series['Hourly Rate'])
mean = np.mean(test_series['Hourly Rate'])
n = len(test_series)




In [30]:
l = np.random.normal(mean, std, n)
test_results = ttest_1samp(l, 30)
display(test_results.statistic)
display(test_results.pvalue)
# pvalue is smaller than 0.05, therefore H0 can be rejected and the hourly wage is significantly different from $ 30/hr

19.70634953167532

1.8503306040664018e-84

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [40]:
# Your code here:

salary = data[['Department','Annual Salary']]
filter_police1 = salary['Department'] == 'POLICE' 
filter_police2 = salary['Department'] == 'POLICE BOARD'
filter_police = filter_police1 | filter_police2
police_salary = salary[filter_police].dropna()

n = len(police_salary)
std = np.std(police_salary['Annual Salary'])
mean = np.mean(police_salary['Annual Salary'])

t_test = (mean - 86000) / (std / np.sqrt(n))
rv = t(df=n-1)
p_value = 1-rv.cdf(t_test)
p_value
# pvalue is smaller than 0.025, therefore H0 can be rejected 

0.0010302480735085995

In [41]:
police_income = np.random.normal(mean, std, n)
test_results = ttest_1samp(police_income, 86000)
display(test_results.statistic)
display(test_results.pvalue/2 )
# pvalue is smaller than 0.025, therefore H0 can be rejected and the salaries are significantly different from $ 86000/year

3.4661766999270993

0.0002647848952064341

Using the `crosstab` function, find the department that has the most hourly workers. 

In [44]:
# Your code here:
data1 = data[['Department', 'Salary or Hourly', 'Hourly Rate']]
data1 = data1.rename(columns = {'Salary or Hourly':'Salary_Hourly', 'Hourly Rate':'Hourly_Rate'})
l = pd.crosstab(data1.Department, data1.Salary_Hourly)
l['Hourly'].max()
l[(l['Hourly'] == 1862)]


Salary_Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [48]:
# Your code here:

hourly_rate = data1[['Department','Hourly_Rate']]
series_hourly = hourly_rate[hourly_rate['Department'] == 'STREETS & SAN'].dropna()
stdv = np.std(series_hourly['Hourly_Rate'])
mean = np.mean(series_hourly['Hourly_Rate'])
stdv, mean




(5.733701558944158, 33.72837808807734)

In [50]:
t_test = (mean - 35) / (stdv / np.sqrt(1862))
t_test
rv = t(df=1862-1)
p_value = rv.cdf(t_test)
p_value / 2

8.149267862408102e-22

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [52]:
# Your code here:

stdv = np.std(series_hourly['Hourly_Rate'])
mean = np.mean(series_hourly['Hourly_Rate'])
t.interval(0.95, df = 1861, loc = mean, scale = stdv)


(22.48321594199933, 44.97354023415535)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [53]:
# Your code here:

data.head()
data_salary_police = data[['Department', 'Annual Salary']].dropna()
data_salary_police_series = data_salary_police['Annual Salary']
mean_police = np.mean(data_salary_police_series)
stdv_police = np.std(data_salary_police_series)
mean_police, stdv_police
n = len(data_salary_police_series)
t.interval(0.95, df = n-1, loc = mean_police, scale = stdv_police)


(45545.5381321311, 128028.46144817176)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:

